In [ ]:
#default_exp utils.utils

In [ ]:
#export
import os
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

import Bert4NER.config as config
import Bert4NER.dataset.dataset as dataset

In [ ]:
#export
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

In [ ]:
#export
def create_path(path):
    if not os.path.exists(path):
        path.mkdir(parents=True, exist_ok=True)
    return path

In [ ]:
#export
def process_data(df):
    sentence_group = df.groupby('Sentence #')
    sentences = sentence_group['Word'].apply(list).values
    tags = sentence_group['encoded_Tag'].apply(list).values
    pos = sentence_group['encoded_POS'].apply(list).values
    return sentences, tags, pos

In [ ]:
#export
def create_loader(text, tags, pos, bs, drop_last=False, sampler=None, ret_dataset=False):
    assert len(text) == len(tags) == len(pos)
    ds = dataset.EntityDataset(texts=text, pos=pos, tags=tags)
    if ret_dataset: return ds
    dl = torch.utils.data.DataLoader(ds, batch_size=bs, drop_last=drop_last,
                                        num_workers=5, sampler=sampler)
    return dl

In [ ]:
#export
def save_label_encoders(le_tag, le_pos):
    if (config.OUTPUT_PATH/'le_tag_encoder.bin').exists() and (config.OUTPUT_PATH/'le_pos_encoder.bin').exists():
        print('encoders already exist')
        return 
    torch.save(le_tag, config.OUTPUT_PATH/'le_tag_encoder.bin')
    torch.save(le_pos, config.OUTPUT_PATH/'le_pos_encoder.bin')
    print('encoders saved!')

In [ ]:
#export
def load_label_encoders():
    if (config.OUTPUT_PATH/'le_tag_encoder.bin').exists() and (config.OUTPUT_PATH/'le_pos_encoder.bin').exists():
        le_tag = torch.load(config.OUTPUT_PATH/'le_tag_encoder.bin')
        le_pos = torch.load(config.OUTPUT_PATH/'le_pos_encoder.bin')
        return le_tag, le_pos
    else:
        print(f"No encoders exist in {str(config.OUTPUT_PATH/'le_tag_encoder.bin')} directory")
        return None